## <strong> 14. 결측 데이터 처리

In [2]:
# 필요한 라이브러리
%pip install missingno
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.


#### <strong> 결측치의 표현: None 객체

In [4]:
# None 객체를 포함한 배열
arr1 = np.array([1, 2, None, 4])
print(arr1.dtype)

# 결측치가 없는 배열
arr2 = np.array([1, 2, 3, 4])
print(arr2.dtype)
# print(arr1.dtype) 했을 때 object로 뜨는걸 볼 수있는데, 배열에 담긴게 여러 자료형이면 이렇게 표기함.
# 생물군계에서 종을 분류하기 미묘한 애들을 원생생물로 분류하듯이 여기서도 미묘한 애들 object로 분류하는 것.

object
int32


In [5]:
# Object 객체는 numpy, pandas 연산과 호환되지 않음 (None은 객체라서 np,pd가 안먹는다 이말임.)
# 오류가 뜨는 것을 볼 수 있다.
arr1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### <strong> 결측치의 표현: NaN 값

In [21]:
# NaN 값을 포함한 배열 객체
arr3 = np.array([1, np.nan, 3, 4])
arr3.dtype
# nan은 float64로 표현함. np나 pd가 작동한단 얘기

dtype('float64')

In [23]:
# NaN 값에 대한 산술 연산
print(1 + np.nan)
print(0 * np.nan)
print(arr3.sum())
# 그냥 nan으로 뜨는걸 볼 수 있다. nan 자체가 Not At Number의 약자임. 숫자가 아니라는 것.
# np나 pd가 진짜 그냥 작동은 한다.( = 오류가 뜨지 않는다.)

nan
nan
nan


In [25]:
# NaN값을 무시하는 집계함수(numpy), nan이 아닌 것들을 집계함.
print(np.nansum(arr3)) 
print(np.nanmax(arr3))
print(np.nanmin(arr3))

8.0
4.0
1.0


In [27]:
# Pandas는 None 객체를 자동으로 NaN값으로 변환한다.

ser = pd.Series([1, np.nan, 2, None])
print(ser)

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64


### <strong> Pandas를 이용한 결측치 처리
---

#### <strong>마스킹을 통한 Null 값 표현
+ ```isnull()```
+ ```notnull()```

In [31]:
# [+] 결측치를 포함한 Series 객체 생성
ser = pd.Series([1, np.nan, 2, None])
ser

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [33]:
# [+] isnull(), isnan이랑 뭐가 다르죠? isnan은 np에서 쓰는거고, isnull은 series에서 사용함.
ser.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [35]:
# [+] notnull()
ser.notnull()

0     True
1    False
2     True
3    False
dtype: bool

#### <strong> 결측치를 포함한 행 또는 열 삭제
+ dropna()

In [38]:
# DataFrame 생성 함수
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}

    return pd.DataFrame(data, ind)

df = make_df('ABC', [0, 1, 2])
df

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [40]:
# [+] NaN 값 추가
df.iloc[0, 1] = np.nan
df.iloc[2, 0] = np.nan
# df.iloc[2, 0] = None (None 값 추가는 nan으로 자동변환을 안해준다.)
df

,A,B,C
0,A0,NaN,C0
1,A1,B1,C1
2,NaN,B2,C2


In [42]:
# [+] 결측치를 포함한 행 삭제
df.dropna(axis = 0)

,A,B,C
1,A1,B1,C1


In [44]:
# [+] 결측치를 포함한 열 삭제
df.dropna(axis=1)

,C
0,C0
1,C1
2,C2


In [46]:
df
# df 출력 해보면 원본이 남아있음.
#즉, dropna는 보기를 생성해주는 녀석임. 써먹을려면 다른 변수 저장을 해야함

# inplace를 쓰면 원본이 변경됨.

,A,B,C
0,A0,NaN,C0
1,A1,B1,C1
2,NaN,B2,C2


In [47]:
# df.dropna(axis = 0, inplace = True) inplace는 옵션값.

In [50]:
# [+] NaN 값 추가
df.iloc[:,2] = np.nan
df

,A,B,C
0,A0,NaN,NaN
1,A1,B1,NaN
2,NaN,B2,NaN


In [52]:
# [+] NaN값이 한 개라도 포함되면 행 또는 열 삭제, 열 삭제시 1, 행 삭제 시 0
# how = 'any' , NaN값이 한개라도 포함되면, 이라는 뜻
# how = 'all' , 모든값이 NaN값이라면, 이라는 뜻
df.dropna(axis = 0, how = 'any')

,A,B,C


In [54]:
# [+] 모든 값이 전부 NaN값인 행 또는 열 삭제, 열 삭제시 1, 행 삭제 시 0
df.dropna(axis = 1,  how = 'all')

,A,B
0,A0,NaN
1,A1,B1
2,NaN,B2


In [56]:
# [+] Non-null(정상으로 측정된) 값의 개수가 임계치보다 작은 행을 삭제, 임계치는 2임.
# thresh 매개변수, 임계치를 설정할 수 있는 option
df.dropna(axis = 0, thresh = 2)

,A,B,C
1,A1,B1,NaN


In [58]:
# [+] Non-null 값의 개수가 임계치보다 작은 열을 삭제
df.dropna( axis = 1 , thresh = 2)

,A,B
0,A0,NaN
1,A1,B1
2,NaN,B2


#### <strong> NaN값을 다른 값을 대체
+ ```fillna()```

In [61]:
# Series 객체 생성
ser = pd.Series([1, np.nan, 2, None, 3],
                index=list('abcde'))
ser

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [63]:
# [+] 결측치를 특정 값을 대체, fillna(), 이거 자주 씀. 이거로 데이터 결측치 문제 해결법 연습해보기.
ser.fillna(0)
# 이미 메서드로 존재한다...

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [65]:
# [+] Forward-fill 방식(= LOCF)
ser.fillna(method = 'ffill') # 아니면 ser.ffill()

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [67]:
# [+] Backward-fill 방식(= NOCB)
ser.fillna(method = 'bfill') # 또는 ser.bfill()

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

### <strong> 결측치 처리 예제: Air Quality 데이터셋

In [70]:
from datetime import datetime
from matplotlib import pyplot as plt

"""
Load AirQualityUCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

C:\Users\K\AppData\Local\Temp\ipykernel_20076\1865652950.py:13: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(input_file,


In [71]:
# 데이터 요약
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9357 entries, 2004-03-10 18:00:00 to 2005-04-04 14:00:00
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CO(GT)         7765 non-null   float64
 1   PT08.S1(CO)    8991 non-null   float64
 2   PT08.S2(NMHC)  8991 non-null   float64
 3   NOx(GT)        7718 non-null   float64
 4   PT08.S3(NOx)   8991 non-null   float64
 5   NO2(GT)        7715 non-null   float64
 6   PT08.S4(NO2)   8991 non-null   float64
 7   PT08.S5(O3)    8991 non-null   float64
 8   RH             8991 non-null   float64
 9   AH             8991 non-null   float64
 10  C6H6(GT)       9357 non-null   float64
dtypes: float64(11)
memory usage: 877.2 KB


In [72]:
# 시각화 설정 옵션
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13
plt.ion()

In [73]:
# 결측 데이터 시각화
import missingno # 별도로 다운 해야 켜짐 %pip install missingno

missingno.matrix(df)

<Axes: >

In [74]:
# 일산화탄소(CO) 시계열 시각화
df['CO(GT)'].plot()

<Axes: xlabel='Datetime'>

In [75]:
# 결측치 대치
imp_locf = df['CO(GT)'].copy().ffill() # LOCF
imp_nocb = df['CO(GT)'].copy().bfill() # NOCB
imp_linear = df['CO(GT)'].copy().interpolate() # 선형 보간
imp_mean = df['CO(GT)'].copy().fillna(df['CO(GT)'].mean())  # 평균값 대체

In [76]:
# K-NN 대치
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)    # default: 2
imp_knn = df.copy().values
imp_knn = imputer.fit_transform(imp_knn)

In [77]:
# K-NN 대치 결과 변환(ndarray -> DataFrame)
imp_df = pd.DataFrame(imp_knn, index=imp_locf.index, columns=df.columns)

In [78]:
# 결측치 처리 결과 시각화
plt.plot(df['CO(GT)'], label='actual', zorder=10)
plt.plot(imp_linear, label='linear interpolation', zorder=3)
plt.plot(imp_nocb, label='nocb', zorder=2)
plt.plot(imp_locf, label='locf', zorder=1)
plt.plot(imp_mean, label='mean substitution', zorder=4)
plt.plot(imp_df['CO(GT)'], label='k-nearest neighbor', zorder=5)
plt.legend(loc='best')
plt.show()